# Epoch Extraction for date/timestamp (follow up of #2149)

In [1]:
# t.timestamp_of_birth - ibis.interval(1, 'D')
# Error: Unknown HTML element <int8> on line 1 
# in label of node 7208238078510608626

In [2]:
from datetime import datetime, date

import ibis
import numpy as np
import pandas as pd
# local
from settings import conf
from utils import cursor2df
from gen_data import salary_date, salary_date_schema

In [3]:
BACKENDS = ['omniscidb', 'postgres']
con = {
    backend: getattr(ibis, backend).connect(**conf[backend]) 
    for backend in BACKENDS
}

In [4]:
df_salary1 = salary_date()
print(df_salary1.dtypes)
df_salary1.info()

last_name                     object
salary                         int64
date_of_birth                 object
timestamp_of_birth    datetime64[ns]
dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4018 entries, 0 to 4017
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   last_name           4018 non-null   object        
 1   salary              4018 non-null   int64         
 2   date_of_birth       4018 non-null   object        
 3   timestamp_of_birth  4018 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 125.7+ KB


In [5]:
sch = salary_date_schema()
sch

ibis.Schema {  
  last_name           string
  salary              float64
  date_of_birth       date
  timestamp_of_birth  timestamp
}

In [6]:
table_name = 'salary'
t = {}
params = {
    'omniscidb': {},
    'postgres': {'if_exists': 'append'}
}

for backend in []:  # BACKENDS
    if hasattr(con[backend], 'load_data'):
        con[backend].drop_table(table_name, force=True)
        con[backend].create_table(table_name, schema=sch)
        con[backend].load_data(table_name, df_salary1, **params[backend])
    t[backend] = con[backend].table(table_name)

TMapDException: TMapDException(error_msg='Exception: Expected double type')

In [ ]:
con['omniscidb'].table(table_name)

In [ ]:
t['omniscidb']

In [ ]:
t['omniscidb'].execute()

In [ ]:
_t = t['omniscidb']

In [ ]:
expr = _t.timestamp_of_birth.epoch().name('tmp')
expr = _t[_t.timestamp_of_birth, expr]
print(expr.compile())
expr.execute();

## Tests

In [ ]:
pd.testing.assert_frame_equal(t['omniscidb'].execute(), t['postgres'].execute())

In [ ]:
df = _t.execute()
df

### 1. Epoch

In [ ]:
for b in BACKENDS:
    try:
        print(' {} '.format(b).center(80, '='))
        expr = t[b].timestamp_of_birth.epoch()
        result = expr.execute().values
        expected = df.timestamp_of_birth.dt.dayofyear.values
        print(expr.compile())
        print('.' * 80)
        print('result:', result)
        print('expected:', expected)
        assert all((result == expected))
    except ibis.common.exceptions.OperationNotDefinedError:
        print('[WW] Not implemented.')
    except AssertionError:
        print('[EE] AssertionError')